In [1]:
from datasets import load_dataset, load_metric

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Загружаем библиотеки.

In [2]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Загружаем huggingface dataset

In [3]:
train, validation = load_dataset("DTU54DL/common-accent", split=["train", "test"])

In [4]:
from datasets import Dataset

Конвертируем датасет в pandas.

In [5]:
df = pd.DataFrame(train)

Визуализируем данные с по крайней мере 3 акцентами.

In [6]:
df2 = df[df.groupby('accent').transform('count').gt(3)['audio']]

In [7]:
print("Labels: ", df2["accent"].unique())
print()
df2.groupby("accent").count()[["audio"]]

Labels:  ['India and South Asia (India, Pakistan, Sri Lanka)' 'Filipino'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Hong Kong English'
 'Singaporean English' 'Austrian' 'Malaysian English'
 'West Indies and Bermuda (Bahamas, Bermuda, Jamaica, Trinidad)'
 'German English,Non native speaker' 'Dutch English' 'Dutch'
 'England English,India and South Asia (India, Pakistan, Sri Lanka),Northern English'
 'Spanish,Foreign,Non-native' 'European,German,Foreign,Non-native'
 'Eastern European English' 'Bangladeshi English' 'Thai'
 'East African Khoja' 'Slavic,polish']



,audio
accent,
Austrian,7
Bangladeshi English,6
Dutch,7
Dutch English,7
East African Khoja,8
Eastern European English,8
"England English,India and South Asia (India, Pakistan, Sri Lanka),Northern English",4
"European,German,Foreign,Non-native",7
Filipino,295


Переименуем соответствующие данные в те, которые соответствуют изначальному датасету

In [8]:
df2["accent"] = df2["accent"].replace(["Singaporean English"], ["chinese"])
df2["accent"] = df2["accent"].replace(["Slavic,polish"], ["slavic"])
df2["accent"] = df2["accent"].replace(["Eastern European English"], ["slavic"])
df2["accent"] = df2["accent"].replace(["Spanish,Foreign,Non-native"], ["spanish"])
df2["accent"] = df2["accent"].replace(["India and South Asia (India, Pakistan, Sri Lanka)"], ["indoiranian"])


/tmp/ipykernel_2524/2353571751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["accent"] = df2["accent"].replace(["Singaporean English"], ["chinese"])
/tmp/ipykernel_2524/2353571751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["accent"] = df2["accent"].replace(["Slavic,polish"], ["slavic"])
/tmp/ipykernel_2524/2353571751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [9]:
print("Labels: ", df2["accent"].unique())
print()
df2.groupby("accent").count()[["audio"]]

Labels:  ['indoiranian' 'Filipino'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Hong Kong English'
 'chinese' 'Austrian' 'Malaysian English'
 'West Indies and Bermuda (Bahamas, Bermuda, Jamaica, Trinidad)'
 'German English,Non native speaker' 'Dutch English' 'Dutch'
 'England English,India and South Asia (India, Pakistan, Sri Lanka),Northern English'
 'spanish' 'European,German,Foreign,Non-native' 'slavic'
 'Bangladeshi English' 'Thai' 'East African Khoja']



,audio
accent,
Austrian,7
Bangladeshi English,6
Dutch,7
Dutch English,7
East African Khoja,8
"England English,India and South Asia (India, Pakistan, Sri Lanka),Northern English",4
"European,German,Foreign,Non-native",7
Filipino,295
"German English,Non native speaker",3024


Обрежем для простоты работы акценты до максимума в 200 аудиоклипов за акцент.

In [10]:
g = df2.groupby('accent')
df3 = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

In [11]:
def sampling_k_elements(group, k=200):
    if len(group) < k:
        return group
    return group.sample(k)

df3 = df2.groupby('accent').apply(sampling_k_elements).reset_index(drop=True)

In [12]:
print("Labels: ", df3["accent"].unique())
print()
df3.groupby("accent").count()[["audio"]]

Labels:  ['Austrian' 'Bangladeshi English' 'Dutch' 'Dutch English'
 'East African Khoja'
 'England English,India and South Asia (India, Pakistan, Sri Lanka),Northern English'
 'European,German,Foreign,Non-native' 'Filipino'
 'German English,Non native speaker' 'Hong Kong English'
 'Malaysian English' 'Southern African (South Africa, Zimbabwe, Namibia)'
 'Thai' 'West Indies and Bermuda (Bahamas, Bermuda, Jamaica, Trinidad)'
 'chinese' 'indoiranian' 'slavic' 'spanish']



,audio
accent,
Austrian,7
Bangladeshi English,6
Dutch,7
Dutch English,7
East African Khoja,8
"England English,India and South Asia (India, Pakistan, Sri Lanka),Northern English",4
"European,German,Foreign,Non-native",7
Filipino,200
"German English,Non native speaker",200


Создаем новый датасет из интересующих нас данных.

In [114]:
df4 = df3[(df3.accent == "chinese") | (df3.accent == "indoiranian") | (df3.accent == "slavic") | (df3.accent == "spanish")].reset_index(drop=True)

In [115]:
df4.groupby("accent").count()[["audio"]]

,audio
accent,
chinese,200
indoiranian,200
slavic,13
spanish,6


In [116]:
df4

,audio,sentence,accent
0,"{'path': 'common_voice_en_20674611.mp3', 'arra...",Bock's description of oak apples is noted in t...,chinese
1,"{'path': 'common_voice_en_21117495.mp3', 'arra...","Then, there were three more generations until ...",chinese
2,"{'path': 'common_voice_en_21275082.mp3', 'arra...",Stress: Nonphonemic stress on final syllable.,chinese
3,"{'path': 'common_voice_en_25220026.mp3', 'arra...",The poverty was especially pronounced in rural...,chinese
4,"{'path': 'common_voice_en_25219952.mp3', 'arra...",The cryptoporticus can be visited through the ...,chinese
...,...,...,...
414,"{'path': 'common_voice_en_31063537.mp3', 'arra...",Many ecological studies have been conducted at...,spanish
415,"{'path': 'common_voice_en_31063629.mp3', 'arra...",He also played running back late in the year.,spanish
416,"{'path': 'common_voice_en_31094505.mp3', 'arra...","Part of the classic ""Traveller"" Alien Modules ...",spanish
417,"{'path': 'common_voice_en_31098543.mp3', 'arra...",Her sister Oksana Kurt is also an internationa...,spanish


Для порядочности изменим порядок колонок аудио, для этого создадим новый датасет

In [33]:
df5=df4
df5["sound_array"]=df5["audio"]

In [49]:
dataset = Dataset.from_pandas(df4)

In [60]:
dataset

Dataset({
    features: ['audio', 'sentence', 'accent'],
    num_rows: 419
})

In [80]:
df4.iloc[1]["audio"]["array"]

array([-1.81898940e-12, -2.91038305e-11, -1.81898940e-11, ...,
       -8.25821189e-07, -1.40898046e-05, -1.23066129e-05])

Визуализируем полученные данные

In [118]:
import random

from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(df4) - 1)
    example = df4.iloc[rand_idx]
    audio = example["audio"]
    label = example["accent"]
    sentence = example["sentence"]
    
    print(f'Sentence: {sentence}')
    print(f'Accent: {label}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    print(f'Array: {audio["array"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Sentence: A cycle of inter-communal violence throughout India followed for several years.
Accent: indoiranian
Shape: (94848,), sampling rate: 16000
Array: [ 2.00088834e-11 -1.45519152e-11 -5.45696821e-12 ... -6.13727025e-09
 -1.61526259e-09 -1.58033799e-08]



Sentence: The complex then serves to anchor actin filaments to the membrane.
Accent: indoiranian
Shape: (91008,), sampling rate: 16000
Array: [ 2.72848411e-12  5.68434189e-12  4.77484718e-12 ...  1.86049147e-06
  3.12254269e-06 -2.69349766e-06]



Sentence: There was no systematic violence nor massacres of ethnic Poles, however.
Accent: indoiranian
Shape: (95040,), sampling rate: 16000
Array: [-3.63797881e-12  1.81898940e-12  0.00000000e+00 ...  4.64305049e-06
  5.36430161e-06  7.59170507e-06]



Sentence: Muncho Lake is situated on the Alaska Highway.
Accent: chinese
Shape: (92544,), sampling rate: 16000
Array: [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00945510e-08
 5.99373493e-08 6.67254341e-08]



Sentence: A new restaurant, balconies, and two new bars were included in the renovation.
Accent: indoiranian
Shape: (137088,), sampling rate: 16000
Array: [1.38243195e-10 1.23691279e-10 1.23691279e-10 ... 4.67608425e-06
 4.56517864e-06 2.30060959e-06]


In [134]:
for var in range(0,419) :
    df4["sound_array"][var] = df4.iloc[var]["audio"]["array"]
    df4["file_name"][var] = df4.iloc[var]["audio"]["path"]

In [122]:
df4["sound_array"] = ""
df4["file_name"] = ""

In [136]:
df4

,audio,sentence,accent,sound_array,file_name
0,"{'path': 'common_voice_en_20674611.mp3', 'arra...",Bock's description of oak apples is noted in t...,chinese,"[2.837623469531536e-10, -2.9831426218152046e-1...",common_voice_en_20674611.mp3
1,"{'path': 'common_voice_en_21117495.mp3', 'arra...","Then, there were three more generations until ...",chinese,"[-1.8189894035458565e-12, -2.9103830456733704e...",common_voice_en_21117495.mp3
2,"{'path': 'common_voice_en_21275082.mp3', 'arra...",Stress: Nonphonemic stress on final syllable.,chinese,"[-1.280568540096283e-09, 2.0954757928848267e-0...",common_voice_en_21275082.mp3
3,"{'path': 'common_voice_en_25220026.mp3', 'arra...",The poverty was especially pronounced in rural...,chinese,"[1.9387045606711586e-26, 9.53196408996653e-26,...",common_voice_en_25220026.mp3
4,"{'path': 'common_voice_en_25219952.mp3', 'arra...",The cryptoporticus can be visited through the ...,chinese,"[1.4823076576950256e-20, 1.3552527156068805e-2...",common_voice_en_25219952.mp3
...,...,...,...,...,...
414,"{'path': 'common_voice_en_31063537.mp3', 'arra...",Many ecological studies have been conducted at...,spanish,"[-7.275957614183426e-11, -1.4551915228366852e-...",common_voice_en_31063537.mp3
415,"{'path': 'common_voice_en_31063629.mp3', 'arra...",He also played running back late in the year.,spanish,"[-3.637978807091713e-12, -2.9103830456733704e-...",common_voice_en_31063629.mp3
416,"{'path': 'common_voice_en_31094505.mp3', 'arra...","Part of the classic ""Traveller"" Alien Modules ...",spanish,"[-7.275957614183426e-12, 4.3655745685100555e-1...",common_voice_en_31094505.mp3
417,"{'path': 'common_voice_en_31098543.mp3', 'arra...",Her sister Oksana Kurt is also an internationa...,spanish,"[-4.3655745685100555e-11, 2.9103830456733704e-...",common_voice_en_31098543.mp3


In [135]:
df4["sound_array"][1]

array([-1.81898940e-12, -2.91038305e-11, -1.81898940e-11, ...,
       -8.25821189e-07, -1.40898046e-05, -1.23066129e-05])

In [137]:
df5 = df4.drop(columns = ['audio', 'sentence'])

In [138]:
df5

,accent,sound_array,file_name
0,chinese,"[2.837623469531536e-10, -2.9831426218152046e-1...",common_voice_en_20674611.mp3
1,chinese,"[-1.8189894035458565e-12, -2.9103830456733704e...",common_voice_en_21117495.mp3
2,chinese,"[-1.280568540096283e-09, 2.0954757928848267e-0...",common_voice_en_21275082.mp3
3,chinese,"[1.9387045606711586e-26, 9.53196408996653e-26,...",common_voice_en_25220026.mp3
4,chinese,"[1.4823076576950256e-20, 1.3552527156068805e-2...",common_voice_en_25219952.mp3
...,...,...,...
414,spanish,"[-7.275957614183426e-11, -1.4551915228366852e-...",common_voice_en_31063537.mp3
415,spanish,"[-3.637978807091713e-12, -2.9103830456733704e-...",common_voice_en_31063629.mp3
416,spanish,"[-7.275957614183426e-12, 4.3655745685100555e-1...",common_voice_en_31094505.mp3
417,spanish,"[-4.3655745685100555e-11, 2.9103830456733704e-...",common_voice_en_31098543.mp3


In [146]:
df6 = pd.DataFrame(columns=['file_name', 'accent'])
print(df6)

Empty DataFrame
Columns: [file_name, accent]
Index: []


In [147]:
df6["file_name"]=df5["file_name"]
df6["accent"]=df5["accent"]
df6

,file_name,accent
0,common_voice_en_20674611.mp3,chinese
1,common_voice_en_21117495.mp3,chinese
2,common_voice_en_21275082.mp3,chinese
3,common_voice_en_25220026.mp3,chinese
4,common_voice_en_25219952.mp3,chinese
...,...,...
414,common_voice_en_31063537.mp3,spanish
415,common_voice_en_31063629.mp3,spanish
416,common_voice_en_31094505.mp3,spanish
417,common_voice_en_31098543.mp3,spanish


In [141]:
df6["sound_array"][1]

array([-1.81898940e-12, -2.91038305e-11, -1.81898940e-11, ...,
       -8.25821189e-07, -1.40898046e-05, -1.23066129e-05])

In [144]:
data_validation = []
for x in range (419) :
    data_validation.append({
            "array": df4.iloc[x]["audio"]["array"]
        }) 

In [148]:
dataset_test_2 = Dataset.from_pandas(df6)

In [149]:
dataset_test_2 = dataset_test_2.add_column("sound_array", data_validation)

In [150]:
for _ in range(5):
    rand_idx = random.randint(0, len(dataset_test_2) - 1)
    example = dataset_test_2[rand_idx]
    audio = example["sound_array"]
    accent = example["accent"]
    file_name = example["file_name"]
    print(f'Accent: {accent}')
    print(f'file_name: {file_name}')
    display(Audio(audio["array"], rate=16000))
    print()

Accent: chinese
file_name: common_voice_en_21947058.mp3



Accent: indoiranian
file_name: common_voice_en_23365546.mp3



Accent: indoiranian
file_name: common_voice_en_24690281.mp3



Accent: indoiranian
file_name: common_voice_en_21948999.mp3



Accent: indoiranian
file_name: common_voice_en_24690281.mp3


In [153]:
dataset_test_2.push_to_hub("TheHolyPacman/train_dataset_2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]